# Problema de Asignación de Armas (Weapon Target Assignment, WTA)

Eduardo Carrasco Vidal, Estudiante Doctorado en Ingeniería de Sistemas Complejos
Fecha actualización: 25 noviembre de 2024

## Definición del Problema
El problema de Asignación de Armas consiste en asignar un conjunto limitado de recursos defensivos (misiles) a múltiples amenazas entrantes (misiles enemigos), con el objetivo de maximizar la probabilidad de neutralización de dichas amenazas. Este problema se resuelve mediante programación lineal entera.

---

## Formulación Matemática

### Variables
- **`x_ij`**: Variable binaria, igual a 1 si el misil `i` se asigna a la amenaza `j`, 0 en caso contrario.
- **`P_ij`**: Probabilidad de impacto del misil `i` contra la amenaza `j`.
- **`N`**: Número total de misiles disponibles.
- **`M`**: Número total de amenazas.

### Función Objetivo
Maximizar la probabilidad total de neutralización:

$$
\text{Maximizar} \quad \sum_{i=1}^{N} \sum_{j=1}^{M} P_{ij} \cdot x_{ij}
$$

### Restricciones
1. **Restricción por misil**: Cada misil solo puede ser asignado a lo sumo a una amenaza:

$$
\sum_{j=1}^{M} x_{ij} \leq 1 \quad \forall i \in \{1, 2, ..., N\}
$$

2. **Restricción por amenaza**: Cada amenaza debe ser atacada por al menos un misil (si es que hay):

$$
\sum_{i=1}^{N} x_{ij} \geq 1 \quad \forall j \in \{1, 2, ..., M\}
$$

3. **Restricción binaria**: Las variables de asignación son binarias:

$$
x_{ij} \in \{0, 1\} \quad \forall i, j
$$

---

## Componentes del Código

### 1. Función `weapon_target_assignment`

**Entradas:**
- `num_ships`: Número de buques disponibles.
- `missiles_per_ship`: Número de misiles por buque.
- `num_threats`: Número de amenazas.
- `hit_probabilities`: Matriz de probabilidades de impacto de los misiles.

**Salidas:**
- `assignment`: Matriz binaria que muestra la asignación óptima de misiles a amenazas.
- `survival_probability`: Probabilidad global de neutralización de las amenazas.

### 2. Función `calculate_combined_probability`
Calcula la probabilidad conjunta de que todas las amenazas sean neutralizadas. Se basa en las probabilidades combinadas de impacto para cada amenaza.

**Fórmula:**
La probabilidad conjunta se calcula como:

$$
P(\text{todas neutralizadas}) = \prod_{j=1}^{M} \left( 1 - \prod_{i=1}^{N} (1 - P_{ij} \cdot x_{ij}) \right)
$$

**Entradas:**
- `assignment`: Matriz de asignación óptima.
- `hit_probabilities`: Matriz de probabilidades de impacto.

**Salida:**
- `combined_probability`: Probabilidad de que todas las amenazas sean neutralizadas.


In [1]:
import pulp
import numpy as np
import time
import random

def weapon_target_assignment(num_ships, missiles_per_ship, num_threats, hit_probabilities):
    """
    Resuelve el problema de asignación de armas para maximizar la neutralización de amenazas.
    """
    # Total de misiles disponibles
    num_missiles = num_ships * missiles_per_ship

    # Crear el modelo
    model = pulp.LpProblem("Weapon_Target_Assignment", pulp.LpMaximize)

    # Variables de decisión
    x = pulp.LpVariable.dicts("x", ((i, j) for i in range(num_missiles) for j in range(num_threats)), cat="Binary")

    # --- Función objetivo ---
    # Maximizar la probabilidad total de neutralización
    objective = pulp.lpSum(hit_probabilities[i, j] * x[i, j] for i in range(num_missiles) for j in range(num_threats))
    model += objective, "Maximize_Neutralization"

    # --- Restricciones ---
    # 1. Cada misil puede ser asignado a lo sumo a una amenaza
    for i in range(num_missiles):
        model += pulp.lpSum(x[i, j] for j in range(num_threats)) <= 1, f"Missile_{i}_Constraint"

    # 2. Cada amenaza debe ser atacada por al menos un misil
    for j in range(num_threats):
        model += pulp.lpSum(x[i, j] for i in range(num_missiles)) >= 1, f"Threat_{j}_Constraint"

    # Resolver el modelo usando el solucionador CBC
    start_time = time.time()
    model.solve(pulp.PULP_CBC_CMD(msg=False))
    end_time = time.time()

    # --- Resultados ---
    # Matriz de asignación
    assignment = np.zeros((num_missiles, num_threats))
    for i in range(num_missiles):
        assigned = False  # Asegurarse de que un misil tenga solo una asignación
        for j in range(num_threats):
            value = pulp.value(x[i, j])
            if value is not None and value > 0.5:
                if not assigned:
                    assignment[i, j] = 1
                    assigned = True
                else:
                    assignment[i, j] = 0

    # Probabilidad de neutralización total
    survival_probability = pulp.value(model.objective)

    return assignment, survival_probability, end_time - start_time

def calculate_combined_probability(assignment, hit_probabilities):
    """
    Calcula la probabilidad conjunta de que todas las amenazas sean neutralizadas.
    """
    num_threats = assignment.shape[1]
    num_missiles = assignment.shape[0]

    combined_probability = 1
    for j in range(num_threats):
        threat_probability = 1
        for i in range(num_missiles):
            if assignment[i, j] > 0.5:  # Si el misil está asignado
                threat_probability *= (1 - hit_probabilities[i, j])
        combined_probability *= (1 - threat_probability)

    return combined_probability

def calculate_probabilities_per_threat(assignment, hit_probabilities):
    """
    Calcula las probabilidades de neutralización para cada amenaza.
    """
    num_threats = assignment.shape[1]
    num_missiles = assignment.shape[0]

    probabilities_per_threat = []

    for j in range(num_threats):
        threat_failure = 1  # Inicializamos la probabilidad de fallo
        for i in range(num_missiles):
            if assignment[i, j] > 0.5:  # Si el misil está asignado
                threat_failure *= (1 - hit_probabilities[i, j])  # Probabilidad de fallo del misil
        # Probabilidad de neutralización
        threat_success = 1 - threat_failure
        probabilities_per_threat.append(threat_success)

    return probabilities_per_threat 

def main():
    # --- Configuración del problema ---
    num_ships = 3  # Número de buques
    missiles_per_ship = 4  # Misiles por buque
    num_threats = 10  # Número de amenazas

    # Probabilidad de impacto personalizada (misiles x amenazas)
    num_missiles = num_ships * missiles_per_ship
    hit_probabilities = np.random.uniform(0.6, 0.8, (num_ships * missiles_per_ship, num_threats))
    hit_probabilities = np.round(hit_probabilities, 2)

    # Mostrar la matriz de probabilidades de impacto
    print("Matriz de probabilidades de impacto (misiles x amenazas):")
    print(hit_probabilities)

    # Resolver el problema
    assignment, survival_probability, exact_time = weapon_target_assignment(
        num_ships, missiles_per_ship, num_threats, hit_probabilities
    )

    # Calcular la probabilidad conjunta
    combined_probability = calculate_combined_probability(assignment, hit_probabilities) * 100

    # Calcular las probabilidades por amenaza
    probabilities_per_threat = calculate_probabilities_per_threat(assignment, hit_probabilities)

    # Mostrar resultados
    print("\nMatriz de asignación óptima (misiles x amenazas):")
    print(assignment)
    print(f"\nTiempo de procesamiento: {exact_time:.4f} segundos")
    print(f"\nProbabilidad de que todas las amenazas sean neutralizadas: {combined_probability:.4f} % ")
    print("\nProbabilidades de neutralización por amenaza:")
    for idx, prob in enumerate(probabilities_per_threat):
        print(f"Amenaza {idx + 1}: {prob:.4f}")

if __name__ == "__main__":
    main()

Matriz de probabilidades de impacto (misiles x amenazas):
[[0.79 0.64 0.73 0.67 0.75 0.7  0.63 0.73 0.61 0.66]
 [0.75 0.78 0.71 0.71 0.73 0.76 0.71 0.69 0.73 0.78]
 [0.69 0.64 0.79 0.68 0.76 0.76 0.72 0.78 0.73 0.62]
 [0.72 0.69 0.6  0.79 0.64 0.68 0.69 0.66 0.7  0.69]
 [0.78 0.72 0.71 0.69 0.69 0.78 0.72 0.66 0.7  0.77]
 [0.77 0.66 0.62 0.66 0.61 0.62 0.79 0.72 0.69 0.72]
 [0.68 0.65 0.75 0.63 0.76 0.74 0.68 0.65 0.79 0.72]
 [0.76 0.69 0.72 0.68 0.7  0.62 0.66 0.71 0.74 0.61]
 [0.65 0.62 0.67 0.69 0.65 0.73 0.74 0.67 0.67 0.7 ]
 [0.68 0.71 0.75 0.75 0.8  0.68 0.6  0.67 0.77 0.72]
 [0.6  0.63 0.66 0.61 0.7  0.63 0.75 0.67 0.6  0.74]
 [0.66 0.77 0.68 0.68 0.75 0.61 0.69 0.75 0.77 0.65]]

Matriz de asignación óptima (misiles x amenazas):
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 

______

In [2]:
import pulp
import numpy as np
import time

def weapon_target_assignment(num_ships, missiles_per_ship, num_threats, hit_probabilities):
    """
    Resuelve el problema de asignación de armas para maximizar la neutralización de amenazas.
    """
    # Total de misiles disponibles
    num_missiles = num_ships * missiles_per_ship

    # Crear el modelo
    model = pulp.LpProblem("Weapon_Target_Assignment", pulp.LpMaximize)

    # Variables de decisión
    x = pulp.LpVariable.dicts("x", ((i, j) for i in range(num_missiles) for j in range(num_threats)), cat="Binary")

    # --- Función objetivo ---
    # Maximizar la probabilidad total de neutralización
    objective = pulp.lpSum(hit_probabilities[i, j] * x[i, j] for i in range(num_missiles) for j in range(num_threats))
    model += objective, "Maximize_Neutralization"

    # --- Restricciones ---
    # 1. Cada misil puede ser asignado a lo sumo a una amenaza
    for i in range(num_missiles):
        model += pulp.lpSum(x[i, j] for j in range(num_threats)) <= 1, f"Missile_{i}_Constraint"

    # 2. Cada amenaza debe ser atacada por al menos un misil
    for j in range(num_threats):
        model += pulp.lpSum(x[i, j] for i in range(num_missiles)) >= 1, f"Threat_{j}_AtLeastOneMissile"

    # Resolver el modelo usando el solucionador CBC
    start_time = time.time()
    model.solve(pulp.PULP_CBC_CMD(msg=False))
    end_time = time.time()

    # --- Resultados ---
    # Matriz de asignación
    assignment = np.zeros((num_missiles, num_threats))
    for i in range(num_missiles):
        assigned = False  # Asegurarse de que un misil tenga solo una asignación
        for j in range(num_threats):
            value = pulp.value(x[i, j])
            if value is not None and value > 0.5:
                if not assigned:
                    assignment[i, j] = 1
                    assigned = True
                else:
                    assignment[i, j] = 0

    # Probabilidad de neutralización total
    survival_probability = pulp.value(model.objective)

    return assignment, survival_probability, end_time - start_time

def calculate_combined_probability(assignment, hit_probabilities):
    """
    Calcula la probabilidad conjunta de que todas las amenazas sean neutralizadas.
    """
    num_threats = assignment.shape[1]
    num_missiles = assignment.shape[0]

    combined_probability = 1
    for j in range(num_threats):
        threat_probability = 1
        for i in range(num_missiles):
            if assignment[i, j] > 0.5:  # Si el misil está asignado
                threat_probability *= (1 - hit_probabilities[i, j])
        combined_probability *= (1 - threat_probability)

    return combined_probability

def calculate_probabilities_per_threat(assignment, hit_probabilities):
    """
    Calcula las probabilidades de neutralización para cada amenaza.
    """
    num_threats = assignment.shape[1]
    num_missiles = assignment.shape[0]

    probabilities_per_threat = []

    for j in range(num_threats):
        threat_failure = 1  # Inicializamos la probabilidad de fallo
        for i in range(num_missiles):
            if assignment[i, j] > 0.5:  # Si el misil está asignado
                threat_failure *= (1 - hit_probabilities[i, j])  # Probabilidad de fallo del misil
        # Probabilidad de neutralización
        threat_success = 1 - threat_failure
        probabilities_per_threat.append(threat_success)

    return probabilities_per_threat 

def main():
    # --- Configuración del problema ---
    num_ships = 2  # Número de buques
    missiles_per_ship = 3  # Misiles por buque
    num_threats = 5  # Número de amenazas

    # Tipos de amenaza
    threat_types = ["misil-la", "misil-ca", "misil-la", "misil-ca", "misil-ca"]
    threat_probabilities = {"misil-la": 0.8, "misil-ca": 0.6}

    # Generar la matriz de probabilidades de impacto según el tipo de amenaza
    num_missiles = num_ships * missiles_per_ship
    hit_probabilities = np.zeros((num_missiles, num_threats))

    for j, threat_type in enumerate(threat_types):
        hit_probabilities[:, j] = threat_probabilities[threat_type]

    # Resolver el problema
    assignment, survival_probability, exact_time = weapon_target_assignment(
        num_ships, missiles_per_ship, num_threats, hit_probabilities
    )

  # Calcular la probabilidad conjunta
    combined_probability = calculate_combined_probability(assignment, hit_probabilities) * 100

    # Calcular las probabilidades por amenaza
    probabilities_per_threat = calculate_probabilities_per_threat(assignment, hit_probabilities)

    # Mostrar resultados
    print("Asignación óptima (misiles x amenazas):")
    print(assignment)
    print(f"\nTiempo de procesamiento: {exact_time:.4f} segundos")
    print(f"\nProbabilidad de que todas las amenazas sean neutralizadas: {combined_probability:.4f} % ")
    print("\nProbabilidades de neutralización por amenaza:")
    for idx, prob in enumerate(probabilities_per_threat):
        print(f"Amenaza {idx + 1} ({threat_types[idx]}): {prob:.4f}")

if __name__ == "__main__":
    main()

Asignación óptima (misiles x amenazas):
[[0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]]

Tiempo de procesamiento: 0.0738 segundos

Probabilidad de que todas las amenazas sean neutralizadas: 16.5888 % 

Probabilidades de neutralización por amenaza:
Amenaza 1 (misil-la): 0.9600
Amenaza 2 (misil-ca): 0.6000
Amenaza 3 (misil-la): 0.8000
Amenaza 4 (misil-ca): 0.6000
Amenaza 5 (misil-ca): 0.6000


# Resolución del Problema Weapon Target Assignment

Se aborda el **problema de asignación de armas (Weapon Target Assignment)**, cuyo objetivo es asignar misiles a amenazas de manera óptima, maximizando la probabilidad de neutralización. Se implementan tres enfoques diferentes para resolver el problema:

1. **Solución exacta** utilizando programación lineal entera (binaria) con el solucionador CBC.
2. **Algoritmo genético** como solución heurística.
3. **Método greedy**, un enfoque aproximado.

---

## Descripción del Problema

Dado:
- \( M \): Número total de misiles disponibles, provenientes de \( S \) buques (\( M = S \times \text{misiles por buque} \)).
- \( T \): Número total de amenazas.
- \( p_{ij} \): Probabilidad de impacto del misil \( i \) contra la amenaza \( j \).

El objetivo es **maximizar la probabilidad total de neutralización** de las amenazas, asignando misiles a amenazas bajo las siguientes restricciones:
1. Cada misil puede ser asignado a lo sumo a una amenaza.
2. Cada amenaza puede recibir a lo sumo un misil.
3. El número total de asignaciones no puede exceder el número de misiles.

---
## Métodos Implementados

### 1. Solución Exacta
Utiliza programación lineal entera (binaria) con el solucionador **CBC**:
- Define **variables de decisión binarias** (\( x[i, j] \)) que indican si el misil \( i \) se asigna a la amenaza \( j \).
- Maximiza la **suma de las probabilidades de impacto** para todas las asignaciones posibles.
- Resuelve el problema respetando las restricciones.

### 2. Algoritmo Genético
Implementa un enfoque evolutivo:
- **Inicialización:** Crea una población inicial de asignaciones válidas.
- **Fitness:** Evalúa cada individuo en la población considerando:
  - La probabilidad combinada de neutralización.
  - Penalizaciones por asignaciones inválidas (como múltiples misiles asignados a una amenaza).
- **Mutación:** Introduce variaciones en las soluciones.
- Evoluciona la población durante un número definido de generaciones, seleccionando los mejores individuos en base al fitness.

### 3. Método Greedy
Utiliza un enfoque directo y rápido:
- Para cada amenaza, asigna el misil con la mayor probabilidad de impacto.
- Continua hasta que se hayan asignado todos los misiles o se cubran todas las amenazas.

---
## Parámetros del Problema

- **Número de buques (\( S \))**: Cantidad de buques involucrados.
- **Misiles por buque**: Capacidad de misiles por buque.
- **Número de amenazas (\( T \))**: Cantidad total de amenazas a neutralizar.
- **Probabilidades de impacto (\( p_{ij} \))**: Valores entre 0.5 y 0.8, generados aleatoriamente.

---
## Resultados Esperados

Cada método proporciona:
1. **Matriz de asignación (\( M \times T \))**: Indica qué misiles se asignan a qué amenazas.
2. **Fitness total**: Probabilidad de neutralización total (para los métodos genético y greedy).
3. **Tiempo de ejecución**: Tiempo requerido para resolver el problema.

---
## Ejecución Principal
La función principal ejecuta los tres métodos y compara los resultados:
1. **Solución Exacta**
2. **Algoritmo Genético**
3. **Método Greedy**

Print:
- Las asignaciones generadas por cada método.
- La probabilidad total de neutralización (fitness).
- El tiempo de procesamiento.

```python
# Ejemplo de ejecución
if __name__ == "__main__":
    main()


In [3]:
import pulp
import numpy as np
import time
import random

def solve_exact(num_ships, missiles_per_ship, num_threats, hit_probabilities):
    """
    Soluciona el problema de forma exacta utilizando CBC (branch and bound / cutting plane).
    Args:
        num_ships: Número de buques.
        missiles_per_ship: Número de misiles por buque.
        num_threats: Número de amenazas.
        hit_probabilities: Matriz de probabilidades de impacto (misiles x amenazas).

    Returns:
        assignment: Matriz de asignación óptima.
        survival_probability: Probabilidad total de neutralización.
        time_elapsed: Tiempo tomado para resolver el problema.
    """
    num_missiles = num_ships * missiles_per_ship  # Total de misiles disponibles

    # Crear el modelo de optimización
    model = pulp.LpProblem("Exact_WTA", pulp.LpMaximize)

    # Definir variables de decisión (binarias)
    x = pulp.LpVariable.dicts(
        "x", ((i, j) for i in range(num_missiles) for j in range(num_threats)), cat="Binary"
    )

    # Función objetivo: Maximizar la probabilidad total de neutralización
    model += pulp.lpSum(hit_probabilities[i, j] * x[i, j] for i in range(num_missiles) for j in range(num_threats))

    # Restricciones
    for i in range(num_missiles):
        # Cada misil se asigna a lo sumo a una amenaza
        model += pulp.lpSum(x[i, j] for j in range(num_threats)) <= 1

    for j in range(num_threats):
        # Cada amenaza recibe a lo sumo un misil
        model += pulp.lpSum(x[i, j] for i in range(num_missiles)) <= 1

    # Limitar el número total de asignaciones al número de misiles disponibles
    model += pulp.lpSum(x[i, j] for i in range(num_missiles) for j in range(num_threats)) <= num_missiles

    # Resolver el modelo usando el solucionador CBC
    start_time = time.time()
    model.solve(pulp.PULP_CBC_CMD(msg=False))
    end_time = time.time()

    # Extraer resultados de asignación
    assignment = np.zeros((num_missiles, num_threats))
    for i in range(num_missiles):
        for j in range(num_threats):
            value = pulp.value(x[i, j])
            if value is not None and value > 0.5:
                assignment[i, j] = 1

    # Obtener la probabilidad total de neutralización
    survival_probability = pulp.value(model.objective)
    return assignment, survival_probability, end_time - start_time

def solve_genetic(num_ships, missiles_per_ship, num_threats, hit_probabilities, 
                  pop_size=50, generations=100, mutation_rate=0.1):
    """
    Soluciona el problema utilizando un algoritmo genético.
    Args:
        num_ships: Número de buques.
        missiles_per_ship: Número de misiles por buque.
        num_threats: Número de amenazas.
        hit_probabilities: Matriz de probabilidades de impacto (misiles x amenazas).
        pop_size: Tamaño de la población.
        generations: Número de generaciones.
        mutation_rate: Tasa de mutación.

    Returns:
        best_individual: Mejor solución encontrada (matriz de asignación).
        best_fitness: Fitness del mejor individuo.
        time_elapsed: Tiempo total tomado.
    """
    num_missiles = num_ships * missiles_per_ship

    def initialize_population():
        """
        Inicializa la población respetando las restricciones del problema.
        """
        population = []
        for _ in range(pop_size):
            individual = np.zeros((num_missiles, num_threats), dtype=int)
            assigned_missiles = set()
            assigned_threats = set()
            for _ in range(min(num_missiles, num_threats)):
                missile = random.choice([m for m in range(num_missiles) if m not in assigned_missiles])
                threat = random.choice([t for t in range(num_threats) if t not in assigned_threats])
                individual[missile, threat] = 1
                assigned_missiles.add(missile)
                assigned_threats.add(threat)
            population.append(individual)
        return population

    def fitness(ind):
        """
        Calcula el fitness de un individuo considerando penalizaciones.
        """
        total_fitness = 0
        penalties = 0

        for j in range(num_threats):
            threat_failure = 1
            missiles_assigned = 0
            for i in range(num_missiles):
                if ind[i, j] == 1:
                    threat_failure *= (1 - hit_probabilities[i, j])
                    missiles_assigned += 1
            total_fitness += (1 - threat_failure)

            # Penalizar si más de un misil está asignado a la misma amenaza
            if missiles_assigned > 1:
                penalties += missiles_assigned - 1

        # Penalizar si un misil está asignado a más de una amenaza
        for i in range(num_missiles):
            threats_assigned = np.sum(ind[i, :])
            if threats_assigned > 1:
                penalties += threats_assigned - 1

        return total_fitness - penalties * 10  # Penalización proporcional

    def mutate(ind):
        """
        Realiza mutaciones asegurando que las restricciones sean respetadas.
        """
        for i in range(num_missiles):
            if random.random() < mutation_rate:
                ind[i, :] = 0
                available_threats = [t for t in range(num_threats) if np.sum(ind[:, t]) == 0]
                if available_threats:
                    threat = random.choice(available_threats)
                    ind[i, threat] = 1
        return ind

    population = initialize_population()
    start_time = time.time()

    # Evolución por generaciones
    for gen in range(generations):
        fitness_values = [fitness(ind) for ind in population]
        new_population = []
        for _ in range(pop_size // 2):
            parent1, parent2 = random.choices(population, weights=fitness_values, k=2)
            child1 = np.copy(parent1)
            child2 = np.copy(parent2)
            child1 = mutate(child1)
            child2 = mutate(child2)
            new_population.extend([child1, child2])
        population = new_population

    end_time = time.time()
    best_individual = max(population, key=fitness)
    best_fitness = fitness(best_individual)
    return best_individual, best_fitness, end_time - start_time

def solve_greedy(num_ships, missiles_per_ship, num_threats, hit_probabilities):
    """
    Soluciona el problema utilizando un método greedy.
    Args:
        num_ships: Número de buques.
        missiles_per_ship: Número de misiles por buque.
        num_threats: Número de amenazas.
        hit_probabilities: Matriz de probabilidades de impacto (misiles x amenazas).

    Returns:
        assignment: Matriz de asignación.
        total_fitness: Fitness total.
        time_elapsed: Tiempo total tomado.
    """
    num_missiles = num_ships * missiles_per_ship
    assignment = np.zeros((num_missiles, num_threats))
    start_time = time.time()

    available_missiles = set(range(num_missiles))
    for j in range(num_threats):
        if not available_missiles:
            break
        best_missile = max(available_missiles, key=lambda i: hit_probabilities[i, j])
        assignment[best_missile, j] = 1
        available_missiles.remove(best_missile)

    end_time = time.time()
    total_fitness = np.sum([1 - np.prod([1 - hit_probabilities[i, j] for i in range(num_missiles) if assignment[i, j] > 0.5]) for j in range(num_threats)])
    return assignment, total_fitness, end_time - start_time

def main():
    # Configuración del problema
    num_ships = 3
    missiles_per_ship = 2
    num_threats = 4
    hit_probabilities = np.random.uniform(0.5, 0.8, (num_ships * missiles_per_ship, num_threats))

    # Resolver con los métodos
    exact_assignment, exact_fitness, exact_time = solve_exact(num_ships, missiles_per_ship, num_threats, hit_probabilities)
    genetic_assignment, genetic_fitness, genetic_time = solve_genetic(num_ships, missiles_per_ship, num_threats, hit_probabilities)
    greedy_assignment, greedy_fitness, greedy_time = solve_greedy(num_ships, missiles_per_ship, num_threats, hit_probabilities)

    # Mostrar resultados
    print("Método Exacto:")
    print("Asignación:\n", exact_assignment)
    print(f"Fitness: {exact_fitness:.4f}, Tiempo: {exact_time:.4f} segundos\n")

    print("Algoritmo Genético:")
    print("Asignación:\n", genetic_assignment)
    print(f"Fitness: {genetic_fitness:.4f}, Tiempo: {genetic_time:.4f} segundos\n")

    print("Método Greedy:")
    print("Asignación:\n", greedy_assignment)
    print(f"Fitness: {greedy_fitness:.4f}, Tiempo: {greedy_time:.4f} segundos\n")

if __name__ == "__main__":
    main()

Método Exacto:
Asignación:
 [[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]]
Fitness: 2.9384, Tiempo: 0.0411 segundos

Algoritmo Genético:
Asignación:
 [[0 1 0 0]
 [0 0 0 1]
 [0 0 0 0]
 [0 0 0 0]
 [1 0 0 0]
 [0 0 1 0]]
Fitness: 2.8195, Tiempo: 0.1108 segundos

Método Greedy:
Asignación:
 [[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]]
Fitness: 2.9384, Tiempo: 0.0000 segundos



# Referencias Bibliográficas:
